In [1]:
import os
import sys

sys.path.append(os.path.abspath('..'))

In [2]:
from typing import Dict, List, Optional, Any
from datetime import datetime

import httpx
from loguru import logger
from smolagents import CodeAgent, LiteLLMModel, Tool, PromptTemplates
import yaml

import src.agent.adapters.tools  as tools

In [3]:
api_base=None

# model_id="ollama/qwen3:4b"
# api_base="http://localhost:11434"

# anthorpic
# model_id="claude-3-7-sonnet-20250219"

#openai
model_id = "gpt-4o"

# google
# model_id = "gemini/gemini-2.0-flash-exp"
# model_id = "gemin/gemini-2.5-flash-preview-04-17"
# model_id = "gemini/gemini-2.5-pro-preview-05-06"

kwargs = {"tools_api_base": "http://localhost:5050",
          "tools_api_limit": 100}

In [4]:
model = LiteLLMModel(model_id=model_id, api_base=api_base)

/Users/steffen/miniconda3/envs/py312/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'fields' has been removed
  warnings.warn(message, UserWarning)


In [5]:
with open("/Users/steffen/agentic_ai/src/agent/prompts/base_prompts.yaml", "r") as file:
    base_prompts = yaml.safe_load(file)


base_prompts['system_prompt'] = base_prompts['system_prompt'].replace("{{current_date}}", datetime.now().strftime("%Y-%m-%d %H:%M:%S"))
prompt_templates = PromptTemplates(**base_prompts)

In [6]:
class FinalAnswerTool(Tool):
    name = "final_answer"
    description = "Provides a final answer to the given problem."
    inputs = {"answer": {"type": "any", "description": "The final answer to the problem"}}
    outputs = {"result": {"type": "any", "description": "The final response to the problem"}}
    output_type = "any"

    def forward(self, answer: Any) -> Any:
        return answer


In [7]:
agent = CodeAgent(tools=[tools.CompareData(**kwargs), 
                         tools.ConvertIdToName(**kwargs),
                         tools.ConvertNameToId(**kwargs),
                         tools.GetData(**kwargs),
                         tools.GetInformation(**kwargs),
                         tools.GetNeighbors(**kwargs),
                         tools.PlotData(**kwargs),
                         FinalAnswerTool()
                        ],  
                  model=model, 
                  stream_outputs=True,
                  additional_authorized_imports=["pandas", "numpy"],
                  prompt_templates=prompt_templates,
                  max_steps=5)

In [8]:
result = agent.run("What is the daily maximum value of PI-P0017 in April 2025?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ What is the daily maximum value of PI-P0017 in April 2025?                                                      │
│                                                                                                                 │
╰─ LiteLLMModel - gpt-4o ─────────────────────────────────────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 1: Convert the asset name to its id                                                                       
  out = name_to_id(["PI-P0017"])                                                                                   
  asset_id = out["asset_ids"]                                                                                      
  print(asset_id)                                                                                                  
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

2025-05-30 10:26:36.499 | INFO     | src.agent.adapters.tools.base:call_api:90 - Fetching data from http://localhost:5050/v1/id_from_name/PI-P0017 with params: {'offset': 0, 'limit': 100}
2025-05-30 10:26:36.675 | INFO     | src.agent.adapters.tools.base:call_api:104 - Reached the last page.


...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Output()

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Step 3: Provide the daily maximum values as the final answer                                                   
  daily_maximum_values = data['data']                                                                              
  final_answer(daily_maximum_values)                                                                               
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Out - Final answer:                      18b04353-839d-40a1-84c1-9b547d09dd80
timestamp                                                
2025-04-01T00:00:00                              1.588889
2025-04-02T00:00:00                              1.556674
2025-04-03T00:00:00                              1.458451
2025-04-04T00:00:00                              1.481194
2025-04-05T00:00:00                              1.378924
2025-04-06T00:00:00                              1.267590
2025-04-07T00:00:00                              1.261139
2025-04-08T00:00:00                              1.290472
2025-04-09T00:00:00                              1.241875
2025-04-10T00:00:00                              1.178278
2025-04-11T00:00:00                              1.025847
2025-04-12T00:00:00                              1.014632
2025-04-13T00:00:00                              0.948687
2025-04-14T00:00:00                              0.888639
2025-04-15T00:00:00                              0.945736
2025-04-16T00:00:00                              0.842243
2025-04-17T00:00:00                              0.785375
2025-04-18T00:00:00                              0.804521
2025-04-19T00:00:00                              0.783035
2025-04-20T00:00:00                              0.699667
2025-04-21T00:00:00                              0.635833
2025-04-22T00:00:00                              0.625757
2025-04-23T00:00:00                              0.619875
2025-04-24T00:00:00                              0.515604
2025-04-25T00:00:00                              0.608785
2025-04-26T00:00:00                              0.500250
2025-04-27T00:00:00                              0.435174
2025-04-28T00:00:00                              0.363062
2025-04-29T00:00:00                              0.378806
2025-04-30T00:00:00                              0.355576

[Step 3: Duration 3.59 seconds| Input tokens: 7,241 | Output tokens: 383]

In [9]:
result

,18b04353-839d-40a1-84c1-9b547d09dd80
timestamp,
2025-04-01T00:00:00,1.588889
2025-04-02T00:00:00,1.556674
2025-04-03T00:00:00,1.458451
2025-04-04T00:00:00,1.481194
2025-04-05T00:00:00,1.378924
2025-04-06T00:00:00,1.267590
2025-04-07T00:00:00,1.261139
2025-04-08T00:00:00,1.290472
2025-04-09T00:00:00,1.241875


## Tests

In [ ]:
tasks = [
    "Can you compare PI-P0017 and PI-P0016 for the first 10 days in 2025?",
    "What assets are next to asset BA100?",
    "Can you create a plot for the adjacent sensors of asset BA101 for 1st January 2025?",
    "What is the id of TI-T0022?",
    "What is the name of asset id c831fadb-d620-4007-bdda-4593038c87f9?",
    "Can you provide me the highest value for June 2025 for TI-T0022?"
]

In [ ]:
result = agent.run(tasks[0])

In [ ]:
result = agent.run(tasks[1])

In [ ]:
result = agent.run(tasks[2])

In [ ]:
base64_string_to_decode = result["plot"]

In [ ]:
import matplotlib.image as mpimg

import base64
import io
import matplotlib.pyplot as plt


In [ ]:
image_bytes = base64.b64decode(base64_string_to_decode)

# 2. Create an in-memory buffer from the bytes
image_buffer = io.BytesIO(image_bytes)

# 3. Load Image from Buffer using Matplotlib
# mpimg.imread can read from a file-like object
img_data = mpimg.imread(image_buffer, format='png') # Specify format if known (e.g., 'png', 'jpeg')

# 4. Display Image using Matplotlib
plt.figure(figsize=(8, 6)) # Optional: Adjust figure size
plt.imshow(img_data)
plt.axis('off')  # Turn off axis numbers and ticks for a cleaner image display
plt.title("Decoded Image")
plt.show()


In [ ]:
result = agent.run(tasks[3])

In [ ]:
result = agent.run(tasks[4])

In [ ]:
result = agent.run(tasks[5])